In [41]:
import asyncio
import json
import os
import base64
from datetime import datetime
from typing import Dict, List, Any
from playwright.async_api import async_playwright

class EncarteGenerator:
	def __init__(self, custom_font_path: str = None):
		# Dimensões fixas
		self.width = 1080
		self.height = 1920
		
		# Margens fixas
		self.header_height = 440
		self.footer_height = 92
		self.content_height = self.height - self.header_height - self.footer_height  # 1400px
  
		self.custom_font_path = custom_font_path
		self.font_family_name = "Futura" 
		self.font_base64 = None
		
		if custom_font_path and os.path.exists(custom_font_path):
				self.font_base64 = self._convert_font_to_base64(custom_font_path)
		
		self.grid_configs = self._initialize_grid_configs()
		self.size_configs = self._initialize_size_configs()
  
	def _convert_font_to_base64(self, font_path: str) -> str:
		"""Converte arquivo TTF para base64 para embedar no HTML"""
		try:
			with open(font_path, "rb") as font_file:
				font_data = font_file.read()
				font_base64 = base64.b64encode(font_data).decode('utf-8')
				print(f"✅ Fonte carregada: {os.path.basename(font_path)}")
				return font_base64
		except Exception as e:
			print(f"❌ Erro ao carregar fonte: {e}")
			return None
    
	def get_font_css(self) -> str:
		"""Gera CSS para a fonte personalizada"""
		if not self.font_base64:
			return ""
		
		return f"""
		@font-face {{
			font-family: '{self.font_family_name}';
			src: url(data:font/truetype;charset=utf-8;base64,{self.font_base64}) format('truetype');
			font-weight: normal;
			font-style: normal;
			font-display: swap;
		}}
		"""
	
	def get_font_family(self) -> str:
		"""Retorna o nome da família da fonte para usar no CSS"""
		if self.font_base64:
			return f"'{self.font_family_name}', Arial, sans-serif"
		else:
			return "Arial, sans-serif"

	def _initialize_grid_configs(self) -> Dict[int, Dict]:
		"""Configurações de grid baseadas no JavaScript fornecido"""
		return {
			4: {
				"gridTemplateColumns": "repeat(2, 1fr)",
				"gridTemplateRows": "repeat(2, 1fr)",
				"gap": "2px",
				"justifyContent": "center",
				"alignItems": "center",
				"justifyItems": "center",
				"items": [
					{"row": 1, "col": 1},
					{"row": 1, "col": 2},
					{"row": 2, "col": 1},
					{"row": 2, "col": 2}
				]
			},
			6: {
				"gridTemplateColumns": "repeat(3, 1fr)",
				"gridTemplateRows": "repeat(2, 1fr)",
				"gap": "2px",
				"justifyContent": "center",
				"alignItems": "center",
				"justifyItems": "center",
				"items": [
					{"row": 1, "col": 1}, {"row": 1, "col": 2}, {"row": 1, "col": 3},
					{"row": 2, "col": 1}, {"row": 2, "col": 2}, {"row": 2, "col": 3}
				]
			},
			9: {
				"gridTemplateColumns": "repeat(3, 1fr)",
				"gridTemplateRows": "repeat(3, 1fr)",
				"gap": "2px",
				"justifyContent": "center",
				"alignItems": "center",
				"justifyItems": "center",
				"items": [
					{"row": 1, "col": 1}, {"row": 1, "col": 2}, {"row": 1, "col": 3},
					{"row": 2, "col": 1}, {"row": 2, "col": 2}, {"row": 2, "col": 3},
					{"row": 3, "col": 1}, {"row": 3, "col": 2}, {"row": 3, "col": 3}
				]
			},
			12: {
				"gridTemplateColumns": "repeat(4, 1fr)",
				"gridTemplateRows": "repeat(3, 1fr)",
				"gap": "2px",
				"justifyContent": "center",
				"alignItems": "center",
				"justifyItems": "center",
				"items": [
					{"row": 1, "col": 1}, {"row": 1, "col": 2}, {"row": 1, "col": 3}, {"row": 1, "col": 4},
					{"row": 2, "col": 1}, {"row": 2, "col": 2}, {"row": 2, "col": 3}, {"row": 2, "col": 4},
					{"row": 3, "col": 1}, {"row": 3, "col": 2}, {"row": 3, "col": 3}, {"row": 3, "col": 4}
				]
			}
		}
	
	def _initialize_size_configs(self) -> Dict[int, Dict]:
		"""Configurações dinâmicas de tamanho baseadas no número total de posições"""
		return {
			4: { # ok
				"font_size_descricao": "26px",
				"font_size_preco_promo": "90px", 
				"font_size_preco_promo_around": "45px",
				"font_size_preco_kg": "24px",
				"font_size_cod_prod": "20px",
				"height_image": "400px",
				"height_description": "100px",
				"height_preco_promo": "120px",
				"height_sub_preco": "36px",
			},
			6: { # ok
				"font_size_descricao": "22px",
				"font_size_preco_promo": "58px",
				"font_size_preco_promo_around": "29px", 
				"font_size_preco_kg": "24px",
				"font_size_cod_prod": "22px",
				"height_image": "330px",
				"height_description": "100px",
				"height_preco_promo": "80px",
				"height_sub_preco": "36px",
			},
			9: { # ok
				"font_size_descricao": "18px",
				"font_size_preco_promo": "56px",
				"font_size_preco_promo_around": "28px",
				"font_size_preco_kg": "20px", 
				"font_size_cod_prod": "18px",
				"height_image": "246px",
				"height_description": "66px",
				"height_preco_promo": "68px",
				"height_sub_preco": "30px",
			},
			12: { # ok
				"font_size_descricao": "16px",
				"font_size_preco_promo": "44px",
				"font_size_preco_promo_around": "22px",
				"font_size_preco_kg": "18px",
				"font_size_cod_prod": "16px",
				"height_image": "246px",
				"height_description": "60px",
				"height_preco_promo": "68px",
				"height_sub_preco": "28px",
			}
		}
 
	def get_grid_config(self, count: int) -> Dict:
		"""Retorna configuração de grid baseada na quantidade de produtos"""
		if count in self.grid_configs:
			return self.grid_configs[count]
		
		# Fallback para configurações não definidas
		return {
			"gridTemplateColumns": "1fr",
			"gridTemplateRows": "1fr", 
			"gap": "2px",
			"justifyContent": "center",
			"alignItems": "center",
			"justifyItems": "center",
			"items": []
		}
	
	def format_price(self, price: str) -> str:
		"""Formata preço para exibição brasileira"""
		try:
			return f"{float(price):.2f}".replace('.', ',')
		except:
			return "0,00"
	
	def truncate_description(self, description: str, max_length: int = 50) -> str:
		"""Trunca descrição se necessário"""
		return description[:max_length] + "..." if len(description) > max_length else description
	
	def get_size_config(self, total_positions: int) -> Dict[str, str]:
		if total_positions in self.size_configs:
			return self.size_configs[total_positions]
		
		# Para grids com mais de 12 posições
		if total_positions > 12:
			# Escala progressiva: quanto mais posições, menores os elementos
			scale_factor = max(0.5, 1 - (total_positions - 12) * 0.05)  # Mínimo 50%
			base_config = self.size_configs[12]  # Usa config de 12 como base
			
			return {
				"font_size_descricao": f"{int(16 * scale_factor)}px",
				"font_size_preco_promo": f"{int(28 * scale_factor)}px",
				"font_size_preco_promo_around": f"{int(16 * scale_factor)}px",
				"font_size_preco_kg": f"{int(14 * scale_factor)}px",
				"font_size_cod_prod": f"{int(12 * scale_factor)}px",
				"height_preco_promo": f"{int(60 * scale_factor)}px",
				"height_image": f"{int(120 * scale_factor)}px",
				"height_description": f"{int(50 * scale_factor)}px"
			}
		
		# Para grids com menos de 4 posições (1, 2, 3)
		elif total_positions < 4:
			# Escala para cima: menos posições = elementos maiores
			scale_factor = 1 + (4 - total_positions) * 0.2  # Máximo 160%
			
			return {
				"font_size_descricao": f"{int(28 * scale_factor)}px",
				"font_size_preco_promo": f"{int(64 * scale_factor)}px",
				"font_size_preco_promo_around": f"{int(32 * scale_factor)}px",
				"font_size_preco_kg": f"{int(24 * scale_factor)}px",
				"font_size_cod_prod": f"{int(18 * scale_factor)}px",
				"height_preco_promo": f"{int(100 * scale_factor)}px",
				"height_image": f"{int(250 * scale_factor)}px",
				"height_description": f"{int(80 * scale_factor)}px"
			}
		
		# Fallback: interpola entre configurações existentes
		else:
			# Encontra as configurações mais próximas e interpola
			configs_sorted = sorted(self.size_configs.keys())
			for i, config_size in enumerate(configs_sorted):
				if total_positions <= config_size:
					return self.size_configs[config_size]
			
			# Se não encontrou, usa a maior configuração
			return self.size_configs[max(configs_sorted)]
 
	def generate_product_card_html(self, data: Dict, produto: Dict, grid_item: Dict, index: int) -> str:
		"""Gera HTML para um card de produto individual"""
		info = produto["produto_info"]
		preco = self.format_price(produto["preco_promo"])
		descricao = self.truncate_description(info["descricao"])
		embalagem = self.truncate_description(info["embalagem"])
		imagem = info["urlImage"]
		destaque = produto.get("destaque", False)
		embalagem_dinamica = produto.get("embalagem_dinamica", "")
  
		shape_un_url = data.get('encarte_info', {}).get('urlTemplateShapeUN', '')
		preco_promo_kilo = produto.get("preco_promo_kilo")
		preco_promo_kilo_format = self.format_price(preco_promo_kilo) if preco_promo_kilo else ""
		codigo_produto = info.get("codigo", "")

		# Pega o número TOTAL de posições do encarte (não a posição individual)
		total_posicoes = data.get("posicoes", len(data.get("produtos", [])))

		# Busca configuração de tamanho baseada no total de posições
		size_config = self.get_size_config(total_posicoes)

		# Extrai todas as configurações
		font_size_descricao = size_config["font_size_descricao"]
		font_size_preco_promo = size_config["font_size_preco_promo"]
		font_size_preco_promo_around = size_config["font_size_preco_promo_around"]
		font_size_preco_kg = size_config["font_size_preco_kg"]
		font_size_cod_prod = size_config["font_size_cod_prod"]
		height_image = size_config["height_image"]
		height_description = size_config["height_description"]
		height_preco_promo = size_config["height_preco_promo"]
		height_sub_preco = size_config["height_sub_preco"]
  
		font_family = self.get_font_family()

		# Posicionamento no grid
		grid_style = f"""
			grid-row: {grid_item['row']};
			grid-column: {grid_item['col']};
			display: flex;
			flex-direction: column;
			align-items: center;
			justify-content: center;
			text-align: center;
		"""
		
		# Visual de destaque
		destaque_html = ""
		if destaque:
			destaque_html = """
			<div style="
				background-color: #fbbf24; 
				position: absolute; 
				top: 10px; 
				right: 10px; 
				width: 60px; 
				height: 4px;
			"></div>
			<div style="
				position: absolute; 
				top: 20px; 
				right: 12px; 
				width: 36px; 
				height: 36px;
				z-index: 10;
			">
   			<svg xmlns="http://www.w3.org/2000/svg" width="36" height="36" viewBox="0 0 36 36" fill=" #fbbf24" stroke=" #fbbf24" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" class="lucide lucide-star-icon lucide-star"><path d="M11.525 2.295a.53.53 0 0 1 .95 0l2.31 4.679a2.123 2.123 0 0 0 1.595 1.16l5.166.756a.53.53 0 0 1 .294.904l-3.736 3.638a2.123 2.123 0 0 0-.611 1.878l.882 5.14a.53.53 0 0 1-.771.56l-4.618-2.428a2.122 2.122 0 0 0-1.973 0L6.396 21.01a.53.53 0 0 1-.77-.56l.881-5.139a2.122 2.122 0 0 0-.611-1.879L2.16 9.795a.53.53 0 0 1 .294-.906l5.165-.755a2.122 2.122 0 0 0 1.597-1.16z"/></svg>
			</div>
   
			<div style="
				background-color: #fbbf24; 
				position: absolute; 
				top: 10px; 
				right: 10px; 
				width: 4px; 
				height: 60px;
			"></div>
			"""
		
		info_adicional_html = ""
		if preco_promo_kilo and preco_promo_kilo != "null" and float(preco_promo_kilo) > 0:
			info_adicional_html = f"""
				<div style="
					width: 95%;
					height: {height_sub_preco};
					display: flex;
					flex-direction: row;
					align-items: flex-start;
					justify-content: space-between;
					box-sizing: border-box;
				">
					<div style="
     				height: 100%;
						font-size: {font_size_preco_kg};
						color: #ffffff;
						font-weight: 500;
						font-family: {font_family};
						background-color: #db1818;
     				border-radius: 12px;
						display: flex;
						align-items: center;
						justify-content: center;
      			padding: 2px 10px;
         		margin-top: 2px;
					">
						R$ {preco_promo_kilo_format} {embalagem_dinamica}
					</div>
					<div style="
						font-size: {font_size_cod_prod};
						color: #666666;
						font-weight: 800;
						font-family: {font_family};
					">
						Cód. {codigo_produto}
					</div>
				</div>
			"""
		else:
			info_adicional_html = f"""
			<div style="
					width: 95%;
					height: {height_sub_preco};
					display: flex;
					flex-direction: row;
					align-items: flex-start;
					justify-content: flex-end;
					padding: 5px 10px;
					box-sizing: border-box;
        ">
					<div style="
						font-size: {font_size_cod_prod};
						color: #666;
						font-weight: 800;
						font-family: {font_family};
					">
						Cód. {codigo_produto}
					</div>
        </div>
			"""
  
		return f"""
		<div style="
			{grid_style}
			background: transparent;
			position: relative;
			overflow: hidden;
			width: 100%;
			height: 100%;
		">
			{destaque_html}
			<div style="
    		width: {height_image};
				height: {height_image};
    		background-color: #bbaacc;
			">
				<img src="{imagem}" alt="produto" style="
					width: 100%;
					height: 100%;
					object-fit: contain;
				" />
			</div>
			<div style="
				height: {height_description};
				width: 95%;
				font-size: {font_size_descricao}; 
				color: #0c0c0c;
				font-weight: 800;
				font-family: {font_family};
				text-align: center;
				display: flex;
				align-items: center;
				justify-content: center;
    		margin-top: 4px;
			">
   			{descricao}
			</div>
			<div style="
   			width: 95%;
				height: {height_preco_promo};
				display: flex;
				flex-direction: row;
				align-items: center;
				justify-content: center;
				background-image: url('{shape_un_url}');
				background-size: 100% 100%;
				background-position: center;
				background-repeat: no-repeat;
			">
				<div style="
					display: flex;
					flex-direction: row;
					align-items: flex-end;
      	">
					<div style="
						font-size: {font_size_preco_promo_around}; 
						color: #fff; 
						font-weight: bold;
						font-family: {font_family};
						padding-bottom: 6px;
						padding-right: 4px;
					">
						R$
					</div>
					<div style="
						font-size: {font_size_preco_promo}; 
						color: #fff; 
						font-weight: bold;
						font-family: {font_family};
					">
						{preco}
					</div>
					<div style="
						font-size: {font_size_preco_promo_around}; 
						color: #fff; 
						font-weight: bold;
						font-family: {font_family};
						padding-bottom: 6px;
      			padding-left: 4px;
					">
						{embalagem}
					</div>
    		</div>
			</div>
			{info_adicional_html}
		</div>
		"""
	
	def generate_header_info(self, data: Dict) -> str:
		"""Gera informações do cabeçalho"""
		try:
			data_inicio = datetime.strptime(data['data_inicio'], "%Y-%m-%d").strftime("%d/%m/%Y")
			data_fim = datetime.strptime(data['data_fim'], "%Y-%m-%d").strftime("%d/%m/%Y")
		except:
			data_inicio = data.get('data_inicio', 'N/A')
			data_fim = data.get('data_fim', 'N/A')
   
		font_family = self.get_font_family()
		
		return f"""
		<div style="
			position: absolute;
			top: 375px;
			left: 50%;
			transform: translateX(-50%);
			display: flex;
			flex-direction: column;
			align-items: center;
			justify-content: center;
			width: 90%;
			font-family: {font_family};
			font-size: 24px;
			text-align: center;
			color: #333;
			z-index: 10;
		">
			<span style="
      	font-weight: 500;
      ">
				Ofertas válidas de <strong style="color: #ff0000;">{data_inicio}</strong> 
				até <strong style="color: #ff0000;">{data_fim}</strong>
			</span>
			<span style="
   			font-size: 24px;
      	font-weight: 500;
			">
				ou enquanto durarem os estoques - <strong style="color: #ff0000;">{data.get('unidade', 'N/A')}</strong>
			</span>
		</div>
		"""
	
	def generate_product_grid(self, data: Dict, produtos: List[Dict], posicoes: int) -> str:
		"""Gera HTML do grid de produtos"""
		if not produtos:
			return "<div>Nenhum produto encontrado</div>"
		
		# Ordena produtos pela posição
		produtos_ordenados = sorted(produtos, key=lambda x: x.get("posicao", 999))
		
		# Pega configuração do grid
		config = self.get_grid_config(posicoes)
		
		# Estilos do grid
		grid_style = f"""
			display: grid;
			grid-template-columns: {config['gridTemplateColumns']};
			grid-template-rows: {config['gridTemplateRows']};
			gap: {config['gap']};
			justify-content: {config.get('justifyContent', 'center')};
			align-items: {config.get('alignItems', 'center')};
			justify-items: {config.get('justifyItems', 'center')};
			width: 100%;
			height: 100%;
			box-sizing: border-box;
			overflow: hidden;
		"""
		
		html = f'<div style="{grid_style}">'
		
		# Gera cada produto na posição correta
		for i, produto in enumerate(produtos_ordenados):
			if i < len(config["items"]):
				grid_item = config["items"][i]
				html += self.generate_product_card_html(data, produto, grid_item, i)
		
		html += "</div>"
		return html
	
	def generate_html(self, data: Dict) -> str:
		"""Gera HTML completo do encarte"""
		posicoes = data.get("posicoes", len(data.get("produtos", [])))
		grid_html = self.generate_product_grid(data, data.get("produtos", []), posicoes)
		header_html = self.generate_header_info(data)
		font_css = self.get_font_css()
		font_family = self.get_font_family()
		
		# URL do background
		background_url = data.get('encarte_info', {}).get('urlTemplateStory', '')
		
		return f"""
		<!DOCTYPE html>
		<html>
		<head>
			<meta charset="utf-8" />
			<meta name="viewport" content="width={self.width}, height={self.height}">
			<style>
				* {{
					box-sizing: border-box;
					margin: 0;
					padding: 0;
				}}
				body {{
					width: {self.width}px;
					height: {self.height}px;
					margin: 0;
					padding: 0;
					background-image: url('{background_url}');
					background-size: cover;
					background-position: center;
					background-repeat: no-repeat;
					font-family: {font_family};
					overflow: hidden;
					position: relative;
					font-family: {font_family};
				}}
				.content-area {{
					position: absolute;
					top: {self.header_height}px;
					left: 0;
					width: 100%;
					height: {self.content_height}px;
					display: flex;
					align-items: center;
					justify-content: center;
				}}
			</style>
		</head>
		<body>
			{header_html}
			<div class="content-area">
				{grid_html}
			</div>
		</body>
		</html>
		"""
	
	async def render_to_image(self, html_content: str, output_path: str = "encarte_resultado.png") -> bool:
		"""Renderiza HTML para imagem usando Playwright"""
		html_file = "temp_encarte.html"
		
		try:
			# Salva HTML temporário
			with open(html_file, "w", encoding="utf-8") as f:
				f.write(html_content)
			
			# Renderiza com Playwright
			async with async_playwright() as p:
				browser = await p.chromium.launch(headless=True)
				page = await browser.new_page(
					viewport={"width": self.width, "height": self.height}
				)
				
				await page.goto(f"file://{os.path.abspath(html_file)}")
				await page.wait_for_load_state('networkidle')
				
				# Aguarda um pouco para garantir que todas as imagens carregaram
				await page.wait_for_timeout(2000)
				
				await page.screenshot(
					path=output_path,
					full_page=False,
					type='png'
				)
				
				await browser.close()
			
			print(f"✅ Encarte Story (1080x1920) gerado: {output_path}")
			return True
			
		except Exception as e:
			print(f"❌ Erro ao gerar encarte: {str(e)}")
			return False
		finally:
			# Remove arquivo temporário
			if os.path.exists(html_file):
				os.remove(html_file)
	
	async def generate_from_json(self, json_data: Dict, output_path: str = None) -> bool:
		"""Gera encarte a partir de dados JSON"""
		try:
			html = self.generate_html(json_data)
			
			if not output_path:
				timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
				output_path = f"encarte_story_{timestamp}.png"
			
			return await self.render_to_image(html, output_path)
			
		except Exception as e:
			print(f"❌ Erro ao processar JSON: {str(e)}")
			return False

async def generate_from_file(json_file_path: str):
	"""Gera encarte a partir de arquivo JSON"""
	try:
		with open(json_file_path, 'r', encoding='utf-8') as file:
			data = json.load(file)
		
		generator = EncarteGenerator("Futura Md BT Bold.ttf")
		success = await generator.generate_from_json(data)
		
		return success
	except Exception as e:
		print(f"❌ Erro ao carregar arquivo: {e}")
		return False

if __name__ == "__main__":
	# Para rodar em Jupyter/Colab
	try:
		import nest_asyncio
		nest_asyncio.apply()
	except:
		pass
	
	
	# Ou para carregar de arquivo:
	asyncio.run(generate_from_file("encarte_data_4.json"))

✅ Fonte carregada: Futura Md BT Bold.ttf
✅ Encarte Story (1080x1920) gerado: encarte_story_20250606_171308.png
